<a href="https://colab.research.google.com/github/tara1enayati/ai-agents-for-beginners/blob/main/New_York_Drivee__py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.7/512.7 kB 39.9 MB/s eta 0:00:00


In [ ]:
!pip install selenium webdriver-manager pandas openpyxl beautifulsoup4

In [ ]:
# دریافت و نصب پکیج رسمی گوگل کروم
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb || apt-get -f install -y

--2025-10-04 12:15:08--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.101.190, 142.250.101.93, 142.250.101.136, ...
Connecting to dl.google.com (dl.google.com)|142.250.101.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120500320 (115M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 114.92M  27.1MB/s    in 5.1s    

2025-10-04 12:15:14 (22.6 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [120500320/120500320]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (141.0.7390.54-1) ...
dpkg: dependency problems prevent configuration of google-chrome-stable:
 google-chrome-stable depends on libvulkan1; however:
  Pack

In [ ]:
!pip install undetected-chromedriver webdriver-manager pandas openpyxl beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47047 sha256=474eab3010e60989cd7161368c99e6db559ada28d069b5733c00dbcecd36ab2a
  Stored in directory: /root/.cache/pip/wheels/c4/f1/aa/9de6cf276210554d91e9c0526864563e850a428c5e76da4914
Successfully built undetected-chromedriver


In [ ]:
!pip install -U fake-useragent


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.3 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
import random
from datetime import datetime
import io
import base64
from IPython.display import display, clear_output, HTML


def scrape_craigslist_json(url):
    """
    استخراج اطلاعات آگهی‌ها با خواندن مستقیم JSON پنهان شده در سورس صفحه.
    """
    headers = {
        'User-Agent': random.choice([
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
            'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)',
            'Mozilla/5.0 (X11; Ubuntu; Linux x86_64)',
            'Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)',
        ])
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"⚠️ خطا در اتصال به سایت: {e}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    json_script_tag = soup.find('script', {'id': 'ld_searchpage_results'})

    if not json_script_tag:
        print("⚠️ تگ JSON پیدا نشد. ممکن است ساختار سایت تغییر کرده باشد.")
        return []

    try:
        json_data = json.loads(json_script_tag.string)
        listings = json_data.get('itemListElement', [])
    except json.JSONDecodeError:
        print("⚠️ خطا در پارس کردن JSON.")
        return []

    extracted_data = []
    for item in listings:
        ad_details = item.get('item', {})

        title = ad_details.get('name', 'بدون عنوان')
        link = ad_details.get('url')
        if link and not link.startswith('http'):
            link = f"https://{url.split('/')[2]}{link}"

        price_info = ad_details.get('offers', {})
        price = price_info.get('price')
        price_currency = price_info.get('priceCurrency', '')
        location_info = price_info.get('availableAtOrFrom', {}).get('address', {})
        location = location_info.get('addressLocality', 'نامشخص')

        extracted_data.append({
            'Title': title,
            'Price': f"{price} {price_currency}" if price else "توافقی",
            'Location': location,
            'Link': link
        })

    return extracted_data


def save_to_excel_and_get_link(dataframe, filename):
    """
    ذخیره فایل در /content (قابل مشاهده در Files Colab) و ساخت لینک دانلود HTML
    """
    save_path = f"/content/{filename}"
    dataframe.to_excel(save_path, index=False, engine='openpyxl')

    # ایجاد لینک دانلود در Colab
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        dataframe.to_excel(writer, index=False, sheet_name='Listings')
    excel_data = output.getvalue()
    b64 = base64.b64encode(excel_data).decode()

    download_link = f'<a href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}" download="{filename}"><b>📥 دانلود فایل اکسل ({datetime.now().strftime("%H:%M:%S")})</b></a>'
    return save_path, download_link


# --- ۳. حلقه اصلی برنامه ---

TARGET_URL = 'https://newyork.craigslist.org/search/sss?query=laptop'
SCRAPE_INTERVAL_SECONDS = 120  # فاصله‌ی زمانی پایه (۲ دقیقه)

print("🤖 ربات آماده به کار است. برای توقف، دکمه Stop را بزنید.")

try:
    while True:
        clear_output(wait=True)
        print(f"⏰ شروع استخراج در: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

        listings_data = scrape_craigslist_json(TARGET_URL)

        if not listings_data:
            print("⚠️ هیچ آگهی‌ای پیدا نشد.")
        else:
            print(f"✅ تعداد {len(listings_data)} آگهی استخراج شد.")

            df = pd.DataFrame(listings_data)
            df['Scraped_At'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"craigslist_laptops_{timestamp}.xlsx"

            save_path, download_link_html = save_to_excel_and_get_link(df, filename)
            display(HTML(download_link_html))

            print(f"💾 فایل ذخیره شد در مسیر: {save_path}")
            print("\n📋 نمونه‌ای از آگهی‌ها:")
            display(df.head())

        # فاصله‌ی زمانی تصادفی بین اجراها
        wait_time = SCRAPE_INTERVAL_SECONDS + random.uniform(-15, 15)
        print(f"\n🔄 اجرای بعدی در حدود {int(wait_time)} ثانیه دیگر...")
        time.sleep(wait_time)

except KeyboardInterrupt:
    print("\n🛑 فرآیند توسط کاربر متوقف شد.")
except Exception as e:
    print(f"❌ خطای غیرمنتظره: {e}")


💾 فایل ذخیره شد در مسیر: /content/craigslist_laptops_20251004_124217.xlsx

📋 نمونه‌ای از آگهی‌ها:


,Title,Price,Location,Link,Scraped_At
0,Black Paw Laptop Bag Black with shoulder strap...,5.00 USD,Ridgewood,None,2025-10-04 12:42:17
1,Acer Aspire 3 laptop - lighly used 256GB i5,100.00 USD,Brooklyn,None,2025-10-04 12:42:17
2,Laptop attache bag Kenneth Cole Reaction,45.00 USD,Brooklyn,None,2025-10-04 12:42:17
3,HP 65W Laptop Power Adapter PA-1650-32HT,12.00 USD,Baldwin,None,2025-10-04 12:42:17
4,Init Laptop Bag,6.00 USD,Baldwin,None,2025-10-04 12:42:17



🔄 اجرای بعدی در حدود 107 ثانیه دیگر...
